In [1]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: cv-3-bitcoin (use `wandb login --relogin` to force relogin)


True

In [2]:
import os
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.7.0 True
11.0
GCC 7.3


In [3]:
# 모듈 import
import mmdet
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset, replace_ImageToTensor)

In [4]:
seed = 42

set_random_seed(seed, deterministic=False)
os.environ['PYTHONHASHSEED'] = str(seed)

In [5]:
from mmcv import Config

# config file 들고오기
baseline_cfg_path = './configs/detectors/detectors_cascade_rcnn_r50_1x_coco.py'
cfg = Config.fromfile(baseline_cfg_path)

In [6]:
model_name = 'detectors_cascade_rcnn_r50_1x_coco.py'
exp = 'multi/albu/cycle'

# Folder to store model logs and weight files
job_folder = f'./working/exp[{exp}]_{model_name}'
cfg.work_dir = job_folder

# Change the wnd username and project name below
wnb_username = 'sunhyuk'
wnb_project_name = 'cv-3-bitcoin'

if not os.path.exists(job_folder):
    os.makedirs(job_folder)

print("Job folder:", job_folder)

Job folder: ./working/exp[multi/albu/cycle]_detectors_cascade_rcnn_r50_1x_coco.py


In [7]:
# dataset settings
dataset_type = 'CocoDataset'
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
root='../../dataset/'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)

In [8]:
albu_train_transforms=[
    dict(
        type='HorizontalFlip',
        p=0.5,
    ),
    dict(
        type='VerticalFlip',
        p=0.5,
    ),
    dict(
        type='RandomBrightnessContrast',
        brightness_limit=[0.1, 0.3],
        contrast_limit=[0.1, 0.3],
        p=0.3),
    dict(
        type='HueSaturationValue',
        p=0.3,
    ),
    dict(
        dict(type="IAAAdditiveGaussianNoise"),
        p=0.3,
    ),
    dict(
        type='Sharpen',
        p=0.3,
    ),
]

cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        #### 이 부분이 multiscale 부분입니다! ####
        type="Resize",
        img_scale=[(1333, 400), (1333, 1200)],
        multiscale_mode="range",
        keep_ratio=True,
    ),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Albu',
        transforms=albu_train_transforms,
        bbox_params=dict(
            type="BboxParams",
            format="pascal_voc",
            label_fields=["gt_labels"],
            min_visibility=0.0,
            filter_lost_elements=True,
        ),
        keymap={"img": "image", "gt_bboxes": "bboxes"},
        update_pad_shape=False,
    ),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels']),
]
# cfg.test_pipeline = [
#     dict(type='LoadImageFromFile'),
#     dict(
#         type='MultiScaleFlipAug',
#         img_scale=(1024, 1024),
#         flip=False,
#         transforms=[
#             dict(type='Resize', keep_ratio=True),
#             dict(type='RandomFlip'),
#             dict(type='Normalize', **img_norm_cfg),
#             dict(type='Pad', size_divisor=32),
#             dict(type='ImageToTensor', keys=['img']),
#             dict(type='Collect', keys=['img']),
#         ])
# ]
cfg.data = dict(
    imgs_per_gpu=4,#####################
    workers_per_gpu=4,
    train=dict(
        type=dataset_type,
        ann_file= root + 'cv_train_1.json',
        img_prefix=root,
        pipeline=cfg.train_pipeline,
        classes = classes),
    val=dict(
        type=dataset_type,
        ann_file= root + 'cv_val_1.json',
        img_prefix=root,
        pipeline=cfg.test_pipeline,
        classes = classes),
    test=dict(
        type=dataset_type,
        ann_file= root + 'test.json',
        img_prefix=root,
        pipeline=cfg.test_pipeline,
        classes = classes),
)


In [9]:
cfg.seed = seed #~~~
cfg.gpu_ids = [0]
#cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'

# cfg.model.roi_head.bbox_head.num_classes = 10 ##클래수 수
# cfg.model.bbox_head.num_classes = 10

cfg.lr_config = dict(
    policy='Cyclic',
    by_epoch = False,
    target_ratio=(1.0,0.005),
    step_ratio_up=0.3,
    cyclic_times=1,
    warmup = 'exp',
    warmup_iters = 5500,
    warmup_ratio = 0.05,
    warmup_by_epoch = False,
)

cfg.evaluation = dict(
    interval=1,
    save_best='auto',
    metric='bbox',
    classwise=True,
)

#
# cfg.optimizer = dict(type='SGD', lr=0.001, momentum=0.9, weight_decay=0.0001)
# cfg.optimizer = dict(type='Adam', lr=3e-4)
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
# cfg.lr_config = dict(
#     policy='step',
#     warmup='linear',
#     warmup_iters=50,
#     warmup_ratio=0.001,
#     step=[8, 11]
# )


cfg.runner.max_epochs = 20

wandb_cfg = dict(config={"epochs": cfg.runner.max_epochs, "optimizer":cfg.optimizer, "model": cfg.model, "img_norm_cfg" : cfg.img_norm_cfg, "train_pipeline" : cfg.train_pipeline, "test_pipeline" : cfg.test_pipeline, "data" : cfg.data, "lr_config" : cfg.lr_config})

cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='WandbLoggerHook',
         interval=10,
         init_kwargs=dict(config=wandb_cfg, project=wnb_username,entity=wnb_project_name, name=f'[{exp}]_{model_name}')
        )
]
cfg.log_config.interval = 10
# cfg.checkpoint_config = 20
print(cfg.pretty_text)


model = dict(
    type='CascadeRCNN',
    backbone=dict(
        type='DetectoRS_ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50'),
        conv_cfg=dict(type='ConvAWS'),
        sac=dict(type='SAC', use_deform=True),
        stage_with_sac=(False, True, True, True),
        output_img=True),
    neck=dict(
        type='RFP',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5,
        rfp_steps=2,
        aspp_out_channels=64,
        aspp_dilations=(1, 3, 6, 1),
        rfp_backbone=dict(
            rfp_inplanes=256,
            type='DetectoRS_ResNet',
            depth=50,
            num_stages=4,
            out_indices=(0, 1, 2, 3),
            frozen_stages=1,
            norm_cfg=dict(type='BN', requires

In [10]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

# dataset 확인
datasets[0]

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!


/opt/conda/envs/mymmdet/lib/python3.7/site-packages/albumentations/imgaug/transforms.py:252: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)



CocoDataset Train dataset with number of images 3677, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 2993  | 1 [Paper]     | 4685  | 2 [Paper pack]  | 681   | 3 [Metal]   | 711   | 4 [Glass]    | 733   |
| 5 [Plastic]       | 2211  | 6 [Styrofoam] | 970   | 7 [Plastic bag] | 3808  | 8 [Battery] | 136   | 9 [Clothing] | 378   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [11]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2022-04-06 18:51:47,367 - mmdet - INFO - load checkpoint from torchvision path: torchvision://resnet50
2022-04-06 18:51:47,641 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

missing keys in source state_dict: layer2.0.conv2.weight_diff, layer2.0.conv2.switch.weight, layer2.0.conv2.switch.bias, layer2.0.conv2.pre_context.weight, layer2.0.conv2.pre_context.bias, layer2.0.conv2.post_context.weight, layer2.0.conv2.post_context.bias, layer2.0.conv2.offset_s.weight, layer2.0.conv2.offset_s.bias, layer2.0.conv2.offset_l.weight, layer2.0.conv2.offset_l.bias, layer2.1.conv2.weight_diff, layer2.1.conv2.switch.weight, layer2.1.conv2.switch.bias, layer2.1.conv2.pre_context.weight, layer2.1.conv2.pre_context.bias, layer2.1.conv2.post_context.weight, layer2.1.conv2.post_context.bias, layer2.1.conv2.offset_s.weight, layer2.1.conv2.offset_s.bias, layer2.1.conv2.offset_l.weight, layer2.1.conv2.offset_l.bias, layer2.2.co

In [12]:
# 모델 학습
train_detector(model, datasets[0], cfg=cfg, distributed=False, validate=True)

2022-04-06 18:51:48,813 - mmdet - WARNING - "imgs_per_gpu" is deprecated in MMDet V2.0. Please use "samples_per_gpu" instead
2022-04-06 18:51:48,814 - mmdet - WARNING - Automatically set "samples_per_gpu"="imgs_per_gpu"=4 in this experiments
2022-04-06 18:51:51,750 - mmdet - INFO - Start running, host: root@642ef2a3bc92, work_dir: /opt/ml/detection/baseline/mmdetection/working/exp[multi/albu/cycle]_detectors_cascade_rcnn_r50_1x_coco.py
2022-04-06 18:51:51,751 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) CyclicLrUpdaterHook                
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) CyclicLrUpdaterHook                
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW     

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


/opt/conda/envs/mymmdet/lib/python3.7/site-packages/mmcv/runner/hooks/evaluation.py:222: UserWarning: runner.meta is None. Creating an empty one.
  warnings.warn('runner.meta is None. Creating an empty one.')


2022-04-06 18:52:18,750 - mmdet - INFO - Epoch [1][10/920]	lr: 1.005e-03, eta: 12:00:32, time: 2.351, data_time: 0.254, memory: 22338, loss_rpn_cls: 0.5493, loss_rpn_bbox: 0.0455, s0.loss_cls: 0.8752, s0.acc: 87.1467, s0.loss_bbox: 0.0467, s1.loss_cls: 0.4972, s1.acc: 79.0079, s1.loss_bbox: 0.0123, s2.loss_cls: 0.2850, s2.acc: 74.8921, s2.loss_bbox: 0.0017, loss: 2.3130, grad_norm: 19.9722
2022-04-06 18:52:41,371 - mmdet - INFO - Epoch [1][20/920]	lr: 1.010e-03, eta: 11:46:33, time: 2.262, data_time: 0.013, memory: 22338, loss_rpn_cls: 0.3606, loss_rpn_bbox: 0.0537, s0.loss_cls: 0.5808, s0.acc: 95.9997, s0.loss_bbox: 0.0778, s1.loss_cls: 0.1472, s1.acc: 97.9341, s1.loss_bbox: 0.0197, s2.loss_cls: 0.0508, s2.acc: 98.5499, s2.loss_bbox: 0.0029, loss: 1.2934, grad_norm: 17.4621
2022-04-06 18:53:04,505 - mmdet - INFO - Epoch [1][30/920]	lr: 1.016e-03, eta: 11:46:51, time: 2.313, data_time: 0.014, memory: 22338, loss_rpn_cls: 0.2026, loss_rpn_bbox: 0.0292, s0.loss_cls: 0.3195, s0.acc: 95.30

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1206/1206, 4.6 task/s, elapsed: 261s, ETA:     0s

...

2022-04-07 07:45:31,166 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.72s).
Accumulating evaluation results...


2022-04-07 07:45:35,831 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.392
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.510
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.422
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.004
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.459
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.538
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.538
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.538
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.043
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.269
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=0.83s).



learning_rate,▁▁▂▂▂▂▃▃▄▅▆▇█████▇▇▇▇▆▆▆▅▅▄▄▄▃▃▂▂▂▂▁▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/grad_norm,▂▆▆▇█▆▇▄▄▄▂▃▃▂▂▂▁▃▁▂▂▃▂▃▁▂▂▄▂▃▁▄▄▃▂▃▃▃▅▃
train/loss,▄▆▆▇█▅█▆▆█▄▆▆▅▄▄▄▆▅▅▆▆▄▆▄▆▃▅▃▄▂▅▃▃▁▃▂▃▃▂
train/loss_rpn_bbox,▄▇█▄█▆▆▅▇▆▃▅▆▇▂▃▃▅▄▅▅▄▄▄▄▅▄▄▄▄▁▆▃▃▂▃▂▃▃▃
train/loss_rpn_cls,█▇▆▆▆▅▅▆▅▇▃▅▄▆▃▂▃▄▂▂▄▃▃▃▄▃▂▃▃▂▁▄▂▁▁▁▁▂▂▁
train/s0.acc,▆▄▃▂▁▄▁▄▃▂▅▃▃▄▅▅▅▃▃▄▃▃▅▃▅▃▅▄▆▄▇▄▆▆█▅▇▇▆▇
train/s0.loss_bbox,▆█▇██▄█▅▆▇▄▆▅▄▃▃▃▅▄▄▄▄▃▅▃▆▃▅▃▄▂▄▃▃▁▃▂▃▂▂
train/s0.loss_cls,▅█▇██▅█▆▆█▄▆▇▅▄▅▄▇▅▆▆▆▄▆▄▆▃▅▃▄▂▄▃▂▁▃▁▂▃▁
train/s1.acc,█▇▄▃▁▃▁▃▂▂▄▂▂▃▄▄▃▃▃▂▂▂▃▂▄▂▄▃▅▄▇▃▅▅▇▅▆▆▅▆
train/s1.loss_bbox,▁▃▅▇█▅█▆▆▇▄▇▅▄▃▃▄▆▅▅▆▅▄▆▃▇▄▆▃▅▂▅▃▄▁▄▂▃▃▃
